In [1]:
!pip install kafka-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 2.3 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip3 install minio
!pip3 install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 2.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.0 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.3 MB/s eta 0:00:0000:0100:01


In [3]:
from datetime import datetime

In [3]:
!pip install clickhouse_driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.8 MB/s eta 0:00:0000:0100:01


In [12]:



from datetime import datetime
from kafka import KafkaConsumer, TopicPartition

from minio import Minio
from minio.error import S3Error
import json
from io import BytesIO
import os

from kafka import KafkaConsumer
TOPIC = 'clients_kafka'
BOOTSTRAP = 'kafka:9092'
ACCESS_KEY = 'minioadmin'
SECRET_KEY = 'minioadmin'


file_path = 'KAFKA/topic=clients_kafka/year=2024/month=8/day=11/2024-08-11 19:31:37.json'
from minio import Minio
BUCKET_NAME = 'bronze-layer'
client = Minio("minio:9000",
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY,
    secure = False
)

data = client.get_object(BUCKET_NAME, file_path)
content = json.load(data)
print(f"Conteúdo do arquivo {file_path}:")
print(content)



Conteúdo do arquivo KAFKA/topic=clients_kafka/year=2024/month=8/day=11/2024-08-11 19:31:37.json:
[{'nome': 'Angela', 'sobrenome': 'Gibson', 'idade': 57, 'email': 'kiarataylor@example.org', 'data_cadastro': '2024-05-05'}, {'nome': 'Angela', 'sobrenome': 'Gibson', 'idade': 57, 'email': 'kiarataylor@example.org', 'data_cadastro': '2024-05-05'}, {'nome': 'Aaron', 'sobrenome': 'Hamilton', 'idade': 36, 'email': 'jennifer89@example.com', 'data_cadastro': '2024-03-02'}, {'nome': 'Angela', 'sobrenome': 'Gibson', 'idade': 57, 'email': 'kiarataylor@example.org', 'data_cadastro': '2024-05-05'}, {'nome': 'Aaron', 'sobrenome': 'Hamilton', 'idade': 36, 'email': 'jennifer89@example.com', 'data_cadastro': '2024-03-02'}, {'nome': 'Nicholas', 'sobrenome': 'Acosta', 'idade': 20, 'email': 'gbarker@example.net', 'data_cadastro': '2024-02-24'}, {'nome': 'Angela', 'sobrenome': 'Gibson', 'idade': 57, 'email': 'kiarataylor@example.org', 'data_cadastro': '2024-05-05'}, {'nome': 'Aaron', 'sobrenome': 'Hamilton', 

In [5]:
!pip install polars

In [14]:
import polars as pl
from datetime import date
content
df_insert = pl.DataFrame(content)
df_insert = df_insert.with_columns(
    pl.col("data_cadastro").str.strptime(pl.Date, format="%Y-%m-%d").alias("data_cadastro")
).with_columns(pl.lit(date(2023, 12, 31)).alias("effective_date"))\
.with_columns(pl.lit(date(2999, 12, 31)).alias("end_date"))\
.with_columns(pl.lit(1).alias("is_current"))
# Exibir o DataFrame
print(df_insert.filter("email = 'kiarataylor@example.org'"))

TypeError: invalid predicate for `filter`: "email = 'kiarataylor@example.org'"

In [19]:
from clickhouse_driver import Client

# Conexão ao servidor ClickHouse
client_ch = Client(
    host='clickhouse',       # Endereço do servidor ClickHouse
    port=9000,              # Porta padrão para conexões TCP
    user='default',         # Usuário (padrão é 'default')
    password='',            # Senha (por padrão, vazio)
    database='default',     # Nome do banco de dados (por padrão, 'default')
)

In [17]:
# Definir os nomes das colunas conforme a tabela do ClickHouse
result = client_ch.execute(f"SELECT * FROM usuarios where is_current = 1")
columns = ['nome', 'sobrenome', 'idade', 'email', 'data_cadastro',  'effective_date', 'end_date', 'is_current']

# Criar o DataFrame Polars a partir dos resultados
df_table = pl.DataFrame(result, schema=columns)

/tmp/ipykernel_423/3037111310.py:5: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df_table = pl.DataFrame(result, schema=columns)


In [76]:
!pip install polars==0.20.23

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 10.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: polars
    Found existing installation: polars 1.4.1
    Uninstalling polars-1.4.1:
      Successfully uninstalled polars-1.4.1


In [83]:
import polars as pl
from datetime import date

def read_table():
    client_ch.execute('''
    CREATE TABLE IF NOT EXISTS dim_clients (
        surrogate_key UUID DEFAULT generateUUIDv4(),
        nome String,
        sobrenome String,
        idade Int32,
        email String,
        data_cadastro Date,
        effective_date Date,
        end_date Date,
        is_current UInt8
    ) ENGINE = MergeTree()
    ORDER BY (nome, sobrenome, effective_date)
    ''')
    result = client_ch.execute(f"SELECT * FROM dim_clients where is_current = 1")
    columns = ['surrogate_key','nome', 'sobrenome', 'idade', 'email', 'data_cadastro',  'effective_date', 'end_date', 'is_current']
    # Criar o DataFrame Polars a partir dos resultados
    df_table = pl.DataFrame(result, schema=columns) 
    return df_table

def insert_dim_clients(content):
    #Criando dataframe polars com as colunas scd2
    df_insert = pl.DataFrame(content)
    df_insert = df_insert.with_columns(
        pl.col("data_cadastro").str.strptime(pl.Date, format="%Y-%m-%d").alias("data_cadastro")
    ).with_columns(pl.lit(date.today()).alias("effective_date"))\
    .with_columns(pl.lit(date(2149, 6, 6)).alias("end_date"))\
    .with_columns(pl.lit(1).alias("is_current"))
    
    #verificando se na base de inserção existem emails repetidos com e mantendo apenas o data_cadastro atual como is_current
    # Ordenar por email, data_cadastro, e qualquer outra coluna para garantir um desempate
    
    # Criar uma contagem cumulativa dentro de cada grupo de email para garantir que apenas o primeiro registro seja 'current'
    df_insert = df_insert.with_columns(
        pl.col("data_cadastro").cum_count(reverse=False).over("email").alias("rank")
    )
    # Atualizar 'is_current' e 'end_date' com base no ranking de email e data cadastro
    df_insert = df_insert.with_columns([
        pl.when(pl.col("rank") == 1)
        .then(pl.lit(1))
        .otherwise(pl.lit(0))
        .alias("is_current"),
    
        pl.when(pl.col("rank") != 1)
        .then(pl.col("effective_date"))
        .otherwise(pl.col("end_date"))
        .alias("end_date")
    ]).drop("rank")

    #lendo a tabela para fazer o scd2
    df_table = read_table()
    if df_table.is_empty():
        data = df_insert.to_dicts()
        
        # Inserir os dados no ClickHouse
        client_ch.execute(
            'INSERT INTO dim_clients (nome, sobrenome, idade, email, data_cadastro, effective_date, end_date, is_current) VALUES',
            [(row['nome'], row['sobrenome'], row['idade'], row['email'], row['data_cadastro'], row['effective_date'], row['end_date'], row['is_current']) for row in data]
        )
        print("Dados inseridos com sucesso")
    else:  
        """
        Nesse caso, eu utilizo o email como chave primaria (poderia ser id, cpf, etc...), faço o join com a tabela para ver os emails repetidos
        que estão sendo inseridos e filtro pela data de cadastro para garantir que os dados repetidos que estamos inserindo
        sao o cadastro mais atualizado.
        
        """
        df_joined = df_insert.join(
            df_table, 
            on="email", 
            how="inner"
        ).filter(
            pl.col("data_cadastro").cast(pl.Date) >= pl.col("data_cadastro_right").cast(pl.Date)
        )
        
        # Linhas a serem inativadas com is_current = 0 e fechamento do end date
        df_update_inactive = df_joined.select([
            pl.col("surrogate_key"),
            pl.col("nome"),
            pl.col("sobrenome"),
            pl.col("idade"),
            pl.col("email"),
            pl.col("data_cadastro_right").alias("data_cadastro"),
            pl.col("effective_date_right").alias("effective_date"),
            pl.col("effective_date").alias("end_date"),
            pl.lit(0).alias("is_current")
        ])
        print('df_update_inactive')
        print(df_update_inactive)
        for row in df_update_inactive.to_dicts():
            client_ch.execute(
                """
                ALTER TABLE dim_clients
                UPDATE end_date = %(end_date)s,
                       is_current = %(is_current)s
                WHERE surrogate_key = %(surrogate_key)s
                """,
                {'end_date': row['end_date'], 'is_current': row['is_current'], 'surrogate_key': row['surrogate_key']}
            )
        
        """ 
        caso onde o cliente a ser inserido ja esta na base mas tem a data de cadastro menor do
        que a já cadastrada, nesse caso, o dado é inserido mas o dado ja persistido
        é mantido como o dado corrente porque tem a data de cadastro maior
        
        """
        df_update_inactive_new  = df_insert.join(
            df_table, 
            on="email", 
            how="inner"
        ).filter(
            pl.col("data_cadastro").cast(pl.Date) < pl.col("data_cadastro_right").cast(pl.Date)
        ).select([
            pl.col("nome"),
            pl.col("sobrenome"),
            pl.col("idade"),
            pl.col("email"),
            pl.col("data_cadastro"),
            pl.col("effective_date"),
            pl.col("effective_date").alias("end_date"),
            pl.lit(0).alias("is_current")
        ])
        print('df_update_inactive_new')
        print(df_update_inactive_new)
        df_insert = df_insert.join(
            df_update_inactive_new, 
            on=["email", "data_cadastro"], 
            how="left"
        ).select([
            pl.col("nome"),
            pl.col("sobrenome"),
            pl.col("idade"),
            pl.col("email"),
            pl.col("data_cadastro"),
            pl.when(pl.col("nome_right").is_not_null()).then(pl.col("effective_date_right")).otherwise(pl.col("effective_date")).alias("effective_date"),
            pl.when(pl.col("nome_right").is_not_null()).then(pl.col("end_date_right")).otherwise(pl.col("end_date")).alias("end_date"),
             pl.when(pl.col("nome_right").is_not_null()).then(pl.col("is_current_right")).otherwise(pl.col("is_current")).alias("is_current"),
        ])
        print('df_insert')
        print(df_insert)
        # Converta o DataFrame para um formato que pode ser enviado ao ClickHouse
        data = df_insert.to_dicts()
        print("dados a serem inseridos:")
        print(data)
        # Inserir os dados no ClickHouse
        client_ch.execute(
            'INSERT INTO dim_clients (nome, sobrenome, idade, email, data_cadastro, effective_date, end_date, is_current) VALUES',
            [(row['nome'], row['sobrenome'], row['idade'], row['email'], row['data_cadastro'], row['effective_date'], row['end_date'], row['is_current']) for row in data]
        )

    

In [84]:
content = [
    {
        "nome": "Robert",
        "sobrenome": "Norton",
        "idade": 50,
        "email": "troy27@example.org",
        "data_cadastro": "2024-07-20"
    },
    {
        "nome": "Robert",
        "sobrenome": "Norton",
        "idade": 50,
        "email": "troy27@example.org",
        "data_cadastro": "2024-07-20"
    },
    {
        "nome": "Robert",
        "sobrenome": "Norton",
        "idade": 50,
        "email": "troy27@example.org",
        "data_cadastro": "2024-03-12"
    }
]

insert_dim_clients(content)

df_update_inactive
shape: (0, 9)
┌──────────────┬──────┬───────────┬───────┬───┬──────────────┬─────────────┬──────────┬────────────┐
│ surrogate_ke ┆ nome ┆ sobrenome ┆ idade ┆ … ┆ data_cadastr ┆ effective_d ┆ end_date ┆ is_current │
│ y            ┆ ---  ┆ ---       ┆ ---   ┆   ┆ o            ┆ ate         ┆ ---      ┆ ---        │
│ ---          ┆ str  ┆ str       ┆ i64   ┆   ┆ ---          ┆ ---         ┆ date     ┆ i32        │
│ object       ┆      ┆           ┆       ┆   ┆ date         ┆ date        ┆          ┆            │
╞══════════════╪══════╪═══════════╪═══════╪═══╪══════════════╪═════════════╪══════════╪════════════╡
└──────────────┴──────┴───────────┴───────┴───┴──────────────┴─────────────┴──────────┴────────────┘
df_update_inactive_new
shape: (3, 8)
┌────────┬───────────┬───────┬──────────────┬──────────────┬─────────────┬────────────┬────────────┐
│ nome   ┆ sobrenome ┆ idade ┆ email        ┆ data_cadastr ┆ effective_d ┆ end_date   ┆ is_current │
│ ---    ┆ ---       

/tmp/ipykernel_581/2672246140.py:22: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df_table = pl.DataFrame(result, schema=columns)


In [20]:

fake.date_this_year().strftime("%Y-%m-%d")

'2024-07-01'

In [75]:
import polars as pl
from datetime import date

# Supondo que você já tenha um DataFrame Polars df_insert com os dados a serem inseridos
data = [
    {"nome": "Robert", "sobrenome": "Norton", "idade": 50, "email": "troy27@example.org", "data_cadastro": "2024-06-17"},
    {"nome": "Robert", "sobrenome": "Norton", "idade": 50, "email": "troy27@example.org", "data_cadastro": "2024-03-12"},
    {"nome": "Robert", "sobrenome": "Norton", "idade": 50, "email": "troy27@example.org", "data_cadastro": "2024-03-12"},
    {"nome": "Robert", "sobrenome": "Norton", "idade": 50, "email": "troy27@example.org", "data_cadastro": "2024-03-06"},
    {"nome": "Robert", "sobrenome": "Norton", "idade": 50, "email": "troy27@example.org", "data_cadastro": "2024-03-06"},
    {"nome": "Robert", "sobrenome": "Norton", "idade": 50, "email": "troy27@example.org", "data_cadastro": "2024-06-17"},
    {"nome": "Robert", "sobrenome": "Norton", "idade": 50, "email": "troy27@example.org", "data_cadastro": "2024-04-06"}
]

df_insert = pl.DataFrame(data)

# Converter a coluna 'data_cadastro' para o tipo Date e adicionar as colunas necessárias
df_insert = df_insert.with_columns([
    pl.col("data_cadastro").str.strptime(pl.Date, format="%Y-%m-%d").alias("data_cadastro"),
    pl.lit(date.today()).alias("effective_date"),
    pl.lit(date(2149, 6, 6)).alias("end_date"),
    pl.lit(1).alias("is_current")
])

# Ordenar por email e data_cadastro em ordem decrescente
df_insert = df_insert.sort(["email", "data_cadastro"], descending=[False, True])

# Atualizar 'is_current' e 'end_date' garantindo que apenas um registro por email/data_cadastro tenha is_current = 1
df_insert = df_insert.group_by("email").agg([
    pl.col("nome"),
    pl.col("sobrenome"),
    pl.col("idade"),
    pl.col("data_cadastro"),
    pl.col("effective_date"),
    pl.col("end_date"),
    pl.first(pl.col("is_current")).alias("is_current")
])

# Exibir o DataFrame atualizado
df_insert


TypeError: invalid input for `col`

Expected `str` or `DataType`, got 'Expr'.

In [13]:
client_ch = Client(
    host='clickhouse',       # Endereço do servidor ClickHouse
    port=9000,              # Porta padrão para conexões TCP
    user='default',         # Usuário (padrão é 'default')
    password='',            # Senha (por padrão, vazio)
    database='default',     # Nome do banco de dados (por padrão, 'default')
)
# Dic
client_ch.execute("SELECT distinct nome, sobrenome, idade, email FROM dim_clients where is_current = 1")

[('Aaron', 'Bishop', 53, 'ihall@example.com'),
 ('Adrian', 'Brown', 38, 'williamlarsen@example.net'),
 ('Alex', 'Olson', 58, 'christopherbass@example.com'),
 ('Becky', 'Peterson', 22, 'nathan14@example.net'),
 ('Benjamin', 'Foster', 30, 'stephanie71@example.com'),
 ('Benjamin', 'Garza', 32, 'hfletcher@example.org'),
 ('Benjamin', 'Mullen', 59, 'lsparks@example.org'),
 ('Benjamin', 'Nichols', 49, 'walkeralyssa@example.com'),
 ('Benjamin', 'Sparks', 63, 'lawrencerobert@example.net'),
 ('Jennifer', 'Lane', 51, 'ogreen@example.org'),
 ('Jennifer', 'Palmer', 60, 'careytammy@example.org'),
 ('Jerry', 'Brown', 35, 'aharris@example.net'),
 ('Jesse', 'Rogers', 66, 'dylan62@example.net'),
 ('Jessica', 'Jordan', 27, 'froth@example.net'),
 ('Jessica', 'Miller', 45, 'brian14@example.org'),
 ('Jessica', 'Rodriguez', 55, 'meganhunt@example.net'),
 ('Jill', 'Mccormick', 26, 'megan09@example.com'),
 ('Jill', 'Ross', 35, 'alicameron@example.org'),
 ('Jillian', 'Russell', 65, 'griffinronald@example.org')